<a href="https://colab.research.google.com/github/kentarooow/GPV_thermal_circuit/blob/main/%E3%81%82%E3%82%8B%E7%B7%AF%E5%BA%A6%E7%B5%8C%E5%BA%A6%E3%81%AB%E3%81%8A%E3%81%91%E3%82%8B1%E6%97%A5%E9%96%93%E3%81%AE%E6%B8%A9%E5%BA%A6%E5%A4%89%E5%8C%96%E3%81%AE%E9%AB%98%E5%BA%A6%E5%88%86%E5%B8%83.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.mkdir('/content/drive/MyDrive/9layer')

#GPVデータの取得


**GPVデータ**は 「Grid Point Value」の略で、大気を3次元の格子に区切り、その一つ一つの格子毎に気温や風速などの値を予測します。

**GPVデータ**はGRIB2というフォーマットで配布されており、数値予報モデルの出力データを格納するための標準的な形式の一つです。

このノートブックでは京都大学生存圏研究所の公開しているGPVデータを取得し、ある緯度経度における1日間の温度変化の高度分布をnumpyの配列として保存します。

#pygribのインストール

**pygrib**はGRIB2形式のデータの読み込みと書き込みを行うためのモジュールです。

pygribを使用することで、Pythonを使って気象データを処理したり、視覚化したりすることができます。

複数のGRIBファイルを結合したり、GRIBファイルをNetCDF形式に変換することもできます。


In [ ]:
pip install pygrib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 100.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 87.4 MB/s eta 0:00:00


#"OS"ライブラリ

オペレーティングシステムとやりとりをするための機能を提供
*   os.getcwd(): 現在の作業ディレクトリを取得
*   os.path.join():パスとファイル名などを結合
*   os.path.basename(): パスのベース名（最後のスラッシュ以降の文字列）を返す


---


#curlコマンドとは

URLを指定してWebサーバーにHTTP/HTTPSリクエストを送信し、レスポンスを受け取るためのコマンド



---


#"subprocess"ライブラリ
別のプログラムを実行し、そのプログラムの出力を扱うための機能を提供
*  subprocess.run(): 引数のコマンドを同期処理で実行



#pygribライブラリのメソッド


*   pygrib.open(): grb形式のファイルを開く gribファイルのイテレータを返す
*   pygrib.open().select(): 引数によって限定されたgribファイルからgribmessageのリストを返す
*   data(): gribmessageのメソッド、緯度経度の範囲を指定することで緯度と経度の格子点とその格子点上のデータを取得

以下のコードの実行でgrb形式のファイルの中身を知ることができます。
```
for grab in grbs:
  print(grab)

```



---




#urlの指定
http://database.rish.kyoto-u.ac.jp/arch/jmadata/data/gpv/original/
の中から必要なデータを探す。
データの名前の見方


1.   月、日（世界標準時刻で指定されているため日本の時刻に対応するためには9時間足す）の順にフォルダに入る
2.   ファイル名中の日付の次の単語を見る
    * GSM: 地球全体の大気を対象とした気象庁の数値予報モデル、Global Spectral Model
    * MSM: 日本及びその近海の大気を対象とした気象庁の数値予報モデル、Meso-Scale Model
    * EPSW: ?
    * GWM: 全球波浪数値予報モデル
3. url_surfに代入

In [ ]:
import os
import subprocess
import pygrib

cwd = os.getcwd()
url_surf = 'http://database.rish.kyoto-u.ac.jp/arch/jmadata/data/gpv/original/2018/10/06/Z__C_RJTD_20181006120000_MSM_GPV_Rjp_L-pall_FH00-15_grib2.bin'
subprocess.run(['curl', '-O', url_surf], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, cwd=cwd)

## GRIB2ファイルを読み込む
file_surf = os.path.join(cwd, os.path.basename(url_surf))
grbs = pygrib.open(file_surf)

In [ ]:
for grab in grbs:
  print(grab)

1:Geopotential Height:gpm (instant):regular_ll:isobaricInhPa:level 100000.0 Pa:fcst time 0 hrs:from 201810061200
2:U component of wind:m s**-1 (instant):regular_ll:isobaricInhPa:level 100000.0 Pa:fcst time 0 hrs:from 201810061200
3:V component of wind:m s**-1 (instant):regular_ll:isobaricInhPa:level 100000.0 Pa:fcst time 0 hrs:from 201810061200
4:Temperature:K (instant):regular_ll:isobaricInhPa:level 100000.0 Pa:fcst time 0 hrs:from 201810061200
5:Vertical velocity:Pa s**-1 (instant):regular_ll:isobaricInhPa:level 100000.0 Pa:fcst time 0 hrs:from 201810061200
6:Relative humidity:% (instant):regular_ll:isobaricInhPa:level 100000.0 Pa:fcst time 0 hrs:from 201810061200
7:Geopotential Height:gpm (instant):regular_ll:isobaricInhPa:level 97500.0 Pa:fcst time 0 hrs:from 201810061200
8:U component of wind:m s**-1 (instant):regular_ll:isobaricInhPa:level 97500.0 Pa:fcst time 0 hrs:from 201810061200
9:V component of wind:m s**-1 (instant):regular_ll:isobaricInhPa:level 97500.0 Pa:fcst time 0 hrs

In [ ]:
import numpy as np
data = np.zeros(144).reshape(9, 16)
for i in range(3):
  for j in range(16):
    levels = [1000,975,950,925,900,850,800,700,600,500,400,300,250,200,150,100] 
    grb = grbs.select(parameterName='Temperature', level=levels[j], forecastTime=3*(i+1))[0]
    values, lats, lons = grb.data(lat1=35.6, lat2=35.7, lon1=139.6, lon2=139.7)
    data[i][j] = values[0][0]

In [ ]:
print(data)

[[299.02020264 297.36727905 295.57479858 293.93093872 292.73724365
  291.89920044 289.99841309 284.05322266 275.82035828 268.64593506
  256.35975647 243.07019043 234.15953064 222.98309326 209.88894653
  199.81057739]
 [298.37493896 296.88424683 295.47293091 294.35848999 293.72558594
  292.340271   290.40808105 284.94869995 277.28599548 268.21868896
  256.36582947 240.38400269 232.62011719 223.24081421 210.64434814
  199.47285461]
 [298.05389404 296.55822754 295.40701294 295.66995239 295.62469482
  293.33831787 290.63372803 285.35983276 277.6751709  267.00827026
  254.2792511  240.36911011 235.04473877 224.64266968 210.10838318
  200.05299377]
 [  0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.
    0.        ]
 [  0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.
    0.           0.           0.      

In [ ]:
import os
import subprocess
import pygrib

# 2018/07/07 9:00(UTC 06:00)の地上データをダウンロードする
cwd = os.getcwd()
url_surf = 'http://database.rish.kyoto-u.ac.jp/arch/jmadata/data/gpv/original/2018/10/07/Z__C_RJTD_20181007000000_MSM_GPV_Rjp_L-pall_FH00-15_grib2.bin'
subprocess.run(['curl', '-O', url_surf], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, cwd=cwd)

## GRIB2ファイルを読み込む
file_surf = os.path.join(cwd, os.path.basename(url_surf))
grbs = pygrib.open(file_surf)

In [ ]:
import numpy as np
for i in range(6):
  for j in range(16):
    levels = [1000,975,950,925,900,850,800,700,600,500,400,300,250,200,150,100] 
    grb = grbs.select(parameterName='Temperature', level=levels[j], forecastTime=3*i)[0]
    values, lats, lons = grb.data(lat1=35.6, lat2=35.7, lon1=139.6, lon2=139.7)
    data[i+3][j] = values[0][0]

In [ ]:
print(data)


[[299.02020264 297.36727905 295.57479858 293.93093872 292.73724365
  291.89920044 289.99841309 284.05322266 275.82035828 268.64593506
  256.35975647 243.07019043 234.15953064 222.98309326 209.88894653
  199.81057739]
 [298.37493896 296.88424683 295.47293091 294.35848999 293.72558594
  292.340271   290.40808105 284.94869995 277.28599548 268.21868896
  256.36582947 240.38400269 232.62011719 223.24081421 210.64434814
  199.47285461]
 [298.05389404 296.55822754 295.40701294 295.66995239 295.62469482
  293.33831787 290.63372803 285.35983276 277.6751709  267.00827026
  254.2792511  240.36911011 235.04473877 224.64266968 210.10838318
  200.05299377]
 [301.27001953 298.93780518 297.34414673 296.53945923 294.81130981
  291.79064941 289.29150391 284.09585571 277.61474609 267.28448486
  254.46786499 241.58604431 233.09812927 222.54615784 211.59223938
  200.58538818]
 [302.97134399 300.44949341 298.20922852 295.96102905 293.74252319
  290.53128052 287.42767334 283.23562622 276.4289093  266.6158447

In [ ]:
np.save(
    "20181007_temp_level",  # ファイル名
    data # 保存したいオブジェクト
)